In [ ]:
from utils import *
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
import pandas as pd
import numpy as np
from collections import *
from sklearn.metrics import *
from sklearn.datasets import *

In [ ]:
import random
from nltk.corpus import stopwords
import re

english_stopwords = stopwords.words('english')
def clearstring(string):
    string = re.sub('[^A-Za-z0-9 ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string if y.strip() not in english_stopwords]
    string = ' '.join(string)
    return string.lower()

def separate_dataset(trainset, ratio = 0.5):
    datastring = []
    datatarget = []
    for i in range(len(trainset.data)):
        data_ = trainset.data[i].split('\n')
        data_ = list(filter(None, data_))
        data_ = random.sample(data_, int(len(data_) * ratio))
        for n in range(len(data_)):
            data_[n] = clearstring(data_[n])
        datastring += data_
        for n in range(len(data_)):
            datatarget.append(trainset.target[i])
    return datastring, datatarget

In [ ]:
trainset = load_files(container_path = '../input/alldata4ai/data_all/data_all/', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

In [ ]:
# build dictionary for the text in order to transform the words to vectors
def build_dataset(words, n_words):
    # count -> most frequently used tokens
    count = [['GO', 0], ['PAD', 1], ['EOS', 2], ['UNK', 3]]
    count.extend(Counter(words).most_common(n_words - 1))
    # word -> int
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    # data -> corpus
    data = list()
    # unk refers to unknown tokens
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    # int -> word
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [ ]:
# one hot matrix for embedding and initialize it
ONEHOT = np.zeros((len(trainset),len(trainset.target_names)))
ONEHOT[np.arange(len(trainset)),] = 1.0

# split the dataset
ONEHOT = np.zeros((len(trainset.data),len(trainset.target_names)))
ONEHOT[np.arange(len(trainset.data)),trainset.target] = 1.0
train_X, test_X, train_Y, test_Y, train_onehot, test_onehot = train_test_split(trainset.data, 
                                                                               trainset.target, 
                                                                               ONEHOT, test_size = 0.2)

In [ ]:
# get all tokens in the text
concat = " ".join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
# get the dictionary
data, count, dictionary, rev_dictionary = build_dataset(concat,vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
#print('Most common words (+UNK)', count[:5])
#print('Sample data', data[:10])

In [ ]:
# release the memory
import gc
del concat
gc.collect()

In [ ]:
# emmmm...release it again in case lack of memory
gc.collect()

In [ ]:
# define model
class Model:
    def __init__(self, size_layer, num_layers, embedded_size,
                 dict_size, dimension_output, learning_rate, attention_size=150):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer,initializer=tf.orthogonal_initializer(),reuse=reuse)
        # for input data, appear with sess.run: feed_dict{} 
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.float32, [None, dimension_output])
        # embedding, random
        # todo: introduce embedding from pre-trained models
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        # add emedding layer
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
        outputs, last_state = tf.nn.dynamic_rnn(rnn_cells, encoder_embedded, dtype = tf.float32)
        attention_w = tf.get_variable("attention_v", [attention_size], tf.float32)
        query = tf.layers.dense(tf.expand_dims(last_state[-1].h, 1), attention_size)
        keys = tf.layers.dense(outputs, attention_size)
        align = tf.reduce_sum(attention_w * tf.tanh(keys + query), [2])
        align = tf.nn.softmax(align)
        outputs = tf.squeeze(tf.matmul(tf.transpose(outputs, [0, 2, 1]),
                                             tf.expand_dims(align, 2)), 2)
        W = tf.get_variable('w',shape=(size_layer, dimension_output),initializer=tf.orthogonal_initializer())
        b = tf.get_variable('b',shape=(dimension_output),initializer=tf.zeros_initializer())
        self.logits = tf.matmul(outputs, W) + b
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
size_layer = 128
num_layers = 2
embedded_size = 128
dimension_output = 2 # “target"
learning_rate = 1e-3 
maxlen = 250 # max number of words in each comment
batch_size = 256

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer,num_layers,embedded_size,vocabulary_size+4,dimension_output,learning_rate)
sess.run(tf.global_variables_initializer())

In [ ]:
# todo: remove explicit for loop
def str_idx(corpus, dic, maxlen, UNK=3):
    X = np.zeros((len(corpus),maxlen))
    rowN = 0
    for row in corpus:
        colN = 0
        for k in row.split(): 
            if colN < maxlen:
                val = dic[k] if k in dic else UNK
                X[rowN, colN]= val
                colN += 1
            else:
                break
        rowN += 1
    return X

In [ ]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(1, (len(train_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(train_X[i:i+batch_size],dictionary,maxlen)
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(test_X[i:i+batch_size],dictionary,maxlen)
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : batch_x, model.Y : test_onehot[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

In [ ]:
# cannot run due to out of memory
# logits = sess.run(model.logits, feed_dict={model.X:str_idx(test_X,dictionary,maxlen)})
# print(classification_report(test_Y, np.argmax(logits,1), target_names = trainset.target_names))